# Test Case LVV-T2219
The objective of this test case is to verify the ComCam communication interface with SAL by testing the commands, events and telemetry defined by the latest version of the XML. This test case will exercise the functionality of the ComCam on the 3rd level of the Summit and meets the following criteria:

    Only requires the ComCam to be operational
    Does not require any communication with other subsystems
    Does require the use of SAL and the EFD
    


In [1]:
from lsst.sitcom import vandv

exec_info = vandv.ExecutionInfo()
print(exec_info)


Executed by isotuela on 2022-08-10T17:00:35.922.
  Running in pillan04 at tucson



# Setup

In [1]:
import asyncio
import logging
import os
import yaml

import astropy.units as u
import numpy as np
import pandas as pd

from astropy.time import Time
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

from lsst.ts.observatory.control.maintel import ComCam
from lsst_efd_client import EfdClient
from lsst.ts import salobj

The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [2]:
os.environ["LSST_DDS_HISTORYSYNC"] = "30"

Setting up logger

In [5]:
logger = logging.getLogger("LVV-T2219")
logger.setLevel(logging.DEBUG)

Getting unique index for Script

In [6]:
logger.info(f'Your UID is {os.getuid()}')
index = os.getuid() * 10 + np.random.randint(0, 9)

logger.info(f'The generated index is {index}')

Make sure DDS Daemon is running and startup Domain

In [7]:
domain = salobj.Domain()

Instantiate script for logging into EFD and start script task

In [8]:
test_message = "LVV-T2219 ComCam Interface to SAL"
script = salobj.Controller("Script", index=index)
await script.start_task

EFD setup

In [8]:
client = vandv.efd.create_efd_client()

ComCam Initialization

In [9]:
comcam = ComCam(domain=domain)
comcam.set_rem_loglevel(40)

In [10]:
await comcam.start_task

[None, None, None]

In [17]:
await comcam.enable()

CancelledError: 

Helper functions: Event callback and telemetry check

In [12]:
def evt_time_callback(evt):
    """Print event and time when callback event is received."""
    evt_time = Time(evt.private_sndStamp, format="unix", scale="tai")
    evt_time.format = "iso"
    evt_name = str(evt.__class__).split('_')[2]
    print(f'\n {evt_name} logevent at {evt_time} is \n \t{evt}')

In [13]:
async def check_telemetry(topic):
    """ Prints the last EFD published telemetry/event """
    tel = await client.select_top_n(topic,'*',num=1)
    tel_time = Time(tel.private_efdStamp, format="unix", scale="utc")
    tel_time.format = "iso"
    print(f'Last {topic} telemetry at {tel_time[0]} UTC (time now {Time.now()}) was \n\n {tel}')

Publish in the EFD that test is starting

In [14]:
script.log.info(f'START- {test_message} -- at {Time(datetime.now())}')

# Start Up

--- 
## 1: ComCam Startup Procedure
LVV-T2239 (1.0) ComCam Startup Procedure

---
## 2: Available Filters 
Verify the CCCamera_logevent_availableFilters event is published to the EFD. 
Note: During the successful start up of the system, this event will be published automatically.

In [109]:
available_filters= comcam.rem.cccamera.evt_availableFilters.get()
time_available_filters=  Time(available_filters.private_sndStamp, format="unix", scale="tai")
time_available_filters.format = "iso"
logger.info(f'Available filters at {time_available_filters} are {available_filters.filterNames}')

---
## 3: CCS Command State 
Initially verify the CCCamera_logevent_ccsCommandState event is publishing to the EFD. 

As there are no commands at the moment, the CCCamera_logevent_ccsCommandState event publishes IDLE.

In [19]:
command_state= comcam.rem.cccamera.evt_ccsCommandState.get()
time_command_state = Time(command_state.private_sndStamp, format="unix", scale="tai")
time_command_state.format = "iso"
logger.info(f'Current Command State Substate at {time_command_state} is \n {command_state}')

---
## 4: Enable Calibration 
While the ccsCommandState is in IDLE, send a CCCamera_command_enableCalibration command.

Note: By default, the CalibrationDetailedState should be in DISABLED. If the CalibrationDetailedState is already in ENABLED, this command will not be accepted.

    Results:
    The CCCamera_logevent_ccsCommandState event publishes BUSY
    The CCCamera_logevent_calibrationDetailedState publishes ENABLED.
    The CCCamera_logevent_ccsConfigured event is published to the EFD noting the time at which the state transition occurred. 
    ** This CCCamera_logevent_ccsConfigured event does not exist, instead comcam.rem.cccamera.logevent_configurationApplied

In [20]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_ccsCommandState.callback = evt_time_callback   

In [21]:
# Enable Calibration
await comcam.rem.cccamera.cmd_enableCalibration.set_start()


 ccsCommandState logevent at 2022-08-10 17:07:47.382 is 
 	private_revCode: d8de8c36, private_sndStamp: 1660151230.38182, private_rcvStamp: 1660151230.3822813, private_seqNum: 45, private_identity: CCCamera, private_origin: 89351361, substate: 2, timestampTransition: 0.0

 calibrationDetailedState logevent at 2022-08-10 17:07:47.383 is 
 	private_revCode: a610ea82, private_sndStamp: 1660151230.3830585, private_rcvStamp: 1660151230.3833776, private_seqNum: 3, private_identity: CCCamera, private_origin: 89351361, substate: 2, timestampTransition: 0.0

 ccsCommandState logevent at 2022-08-10 17:07:47.384 is 
 	private_revCode: d8de8c36, private_sndStamp: 1660151230.3839157, private_rcvStamp: 1660151230.3842075, private_seqNum: 46, private_identity: CCCamera, private_origin: 89351361, substate: 1, timestampTransition: 0.0


In [22]:
configuration_applied = comcam.rem.cccamera.evt_configurationApplied.get()
time_configuration_applied = Time(configuration_applied.private_sndStamp, format="unix", scale="tai")
time_configuration_applied.format = "iso"
logger.info(f'Current configuration Applied at {time_configuration_applied} is \n {configuration_applied}')

In [23]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = None
comcam.rem.cccamera.evt_ccsCommandState.callback = None   

---
## 5: Clear Command
While the CalibrationDetailedState is in ENABLED,, send a CCCamera_command_clear command.

    Deviation: The CCCamera_command_clear no longer operational as part of the latest XML release (v12.0.0).
    The detector is automatically cleared every time exposure is taken. Therefore, no separate clearing is needed and the   CCCamera_logevent_RaftsDetailedState is not expected to be published as a result of this command. This step was left in order to verify that the CCCamera_command_clear will not do anything. The CLEARING state will show up after an exposure is taken (See the Image Test below).


    Results:
    The command is accepted but publishes no events    

In [24]:
cmd = await comcam.rem.cccamera.cmd_clear.set_start(nClears=2)
logger.info(cmd)

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=47224006, ack=<SalRetCode.CMD_FAILED: -302>, error=0, result='Error : Timeout waiting for state: QUIESCENT')

---
## 6: Focal Plane Summary Info
Verify the CCCamera_logevent_focalPlaneSummaryInfo event is published to the EFD.

In [25]:
focal_plane_summary = await comcam.rem.cccamera.evt_focalPlaneSummaryInfo.aget()
time_focal_plane_summary = Time(focal_plane_summary.private_sndStamp, format="unix", scale="tai")
time_focal_plane_summary.format = "iso"
logger.info(f'Focal plane summary at {time_focal_plane_summary}' 
            f' is \n {focal_plane_summary}')

# Image Test

---
## 7: Start Image
While the CalibrationDetailedState is in ENABLED, send a CCCamera_command_startImage command with the following parameters:

                            shutter: True
                            sensors
                            keyValueMap:
                                config/requiredKeywords = [groupId, imageType]
                            obsNote: Any arbitrary string
                            timeout: 30s
    
    Results:  
    The CCCamera_logevent_calibrationDetailedState event publishes INTEGRATING
    The CCCamera_logevent_shutterDetailedState event publishes to the EFD updating froom CLOSED to OPEN
    A CCCamera_logevent_endShutterOpen event is published at the time the shutter opens
    A CCCamera_logevent_startReadout event is published
    The CCCamera_logevent_RaftsDetailedState event published to the EFD shows it is in the CLEARING state.
    
Beware: The next two steps (discardRows and endImage) need to be run while the camera is INTEGRATING (30 s timeout)

In [26]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_shutterDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_startReadout.callback = evt_time_callback
comcam.rem.cccamera.evt_raftsDetailedState.callback = evt_time_callback

In [27]:
keyValueMap = f'groupId:{Time.now().isot},imageType:ENGTEST'

In [28]:
# Set parameters
comcam.rem.cccamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=30, obsNote='Test')

True

In [29]:
# Start image
await comcam.rem.cccamera.cmd_startImage.start()


 raftsDetailedState logevent at 2022-08-10 17:08:43.311 is 
 	private_revCode: d324a9d8, private_sndStamp: 1660151286.3110867, private_rcvStamp: 1660151286.3114235, private_seqNum: 10, private_identity: CCCamera, private_origin: 89351361, substate: 2, timestampTransition: 0.0

 raftsDetailedState logevent at 2022-08-10 17:08:43.413 is 
 	private_revCode: d324a9d8, private_sndStamp: 1660151286.4132493, private_rcvStamp: 1660151286.413685, private_seqNum: 11, private_identity: CCCamera, private_origin: 89351361, substate: 3, timestampTransition: 0.0



 shutterDetailedState logevent at 2022-08-10 17:08:43.417 is 
 	private_revCode: 0ebe7cbe, private_sndStamp: 1660151286.4165206, private_rcvStamp: 1660151286.4168515, private_seqNum: 4, private_identity: CCCamera, private_origin: 89351361, substate: 4, timestampTransition: 0.0

 calibrationDetailedState logevent at 2022-08-10 17:08:43.417 is 
 	private_revCode: a610ea82, private_sndStamp: 1660151286.4171152, private_rcvStamp: 1660151286.4174254, private_seqNum: 4, private_identity: CCCamera, private_origin: 89351361, substate: 3, timestampTransition: 0.0

 shutterDetailedState logevent at 2022-08-10 17:08:44.395 is 
 	private_revCode: 0ebe7cbe, private_sndStamp: 1660151287.3954487, private_rcvStamp: 1660151287.3956983, private_seqNum: 5, private_identity: CCCamera, private_origin: 89351361, substate: 2, timestampTransition: 0.0

 endShutterOpen logevent at 2022-08-10 17:08:44.395 is 
 	private_revCode: dc9d3865, private_sndStamp: 1660151287.395229, private_rcvStamp: 1660151287.3956435

----
## 8: Discard Rows 
While the CalibrationDetailedState is in INTEGRATING, send a CCCamera_command_discardRows command of 2 rows.

In [30]:
cmd = await comcam.rem.cccamera.cmd_discardRows.set_start(2)
logger.info(cmd)

--- 
## 9: End Image 
Before the specified timeout, send a CCCamera_command_endImage command.

Note: If the CCCamera_command_endImage command isn't sent before the specified timeout parameter of the previous command, the shutter will close and the command will not be accepted.

    Results:
    The shutters close 
    A CCCamera_logevent_imageReadoutParameters event is published
    The CCCamera_logevent_shutterDetailedState event publishes to the EFD updating froom OPEN to CLOSED
    A CCCamera_logevent_endShutterClose event is published at the time the shutter close
    A CCCamera_logevent_endReadout event is published

In [31]:
comcam.rem.cccamera.evt_imageReadoutParameters.callback = evt_time_callback
comcam.rem.cccamera.evt_shutterDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_endReadout.callback = evt_time_callback

In [32]:
await comcam.rem.cccamera.cmd_endImage.start()


 shutterDetailedState logevent at 2022-08-10 17:08:57.314 is 
 	private_revCode: 0ebe7cbe, private_sndStamp: 1660151300.3139791, private_rcvStamp: 1660151300.3141387, private_seqNum: 6, private_identity: CCCamera, private_origin: 89351361, substate: 3, timestampTransition: 0.0

 endShutterClose logevent at 2022-08-10 17:08:58.294 is 
 	private_revCode: 1a6c8bc0, private_sndStamp: 1660151301.294318, private_rcvStamp: 1660151301.2946799, private_seqNum: 1, private_identity: CCCamera, private_origin: 89351361, timestampTransition: 1660151301.2926753

 shutterDetailedState logevent at 2022-08-10 17:08:58.296 is 
 	private_revCode: 0ebe7cbe, private_sndStamp: 1660151301.2962174, private_rcvStamp: 1660151301.2966664, private_seqNum: 7, private_identity: CCCamera, private_origin: 89351361, substate: 1, timestampTransition: 0.0

 startReadout logevent at 2022-08-10 17:08:59.403 is 
 	private_revCode: 0bfffb9a, private_sndStamp: 1660151302.4029078, private_rcvStamp: 1660151302.403161, private_


 raftsDetailedState logevent at 2022-08-10 17:09:01.398 is 
 	private_revCode: d324a9d8, private_sndStamp: 1660151304.398247, private_rcvStamp: 1660151304.3984876, private_seqNum: 13, private_identity: CCCamera, private_origin: 89351361, substate: 5, timestampTransition: 0.0

 endReadout logevent at 2022-08-10 17:09:01.398 is 
 	private_revCode: 82211edd, private_sndStamp: 1660151304.3979392, private_rcvStamp: 1660151304.3983154, private_seqNum: 2, private_identity: CCCamera, private_origin: 89351361, additionalKeys: groupId:imageType, additionalValues: 2022-08-10T17\:07\:25.258:ENGTEST, imagesInSequence: 1, imageName: CC_O_20220810_000001, imageIndex: 2, imageSource: CC, imageController: O, imageDate: 20220810, imageNumber: 1, timestampAcquisitionStart: 1660097196.964005, requestedExposureTime: 0.0, timestampEndOfReadout: 1660151304.3955965

 raftsDetailedState logevent at 2022-08-10 17:09:05.399 is 
 	private_revCode: d324a9d8, private_sndStamp: 1660151308.3986669, private_rcvStamp:

In [33]:
comcam.rem.cccamera.evt_imageReadoutParameters.callback = None
comcam.rem.cccamera.evt_shutterDetailedState.callback = None
comcam.rem.cccamera.evt_endShutterClose.callback = None
comcam.rem.cccamera.evt_endReadout.callback = None

In [34]:
# Closing callbacks from startImage.
comcam.rem.cccamera.evt_calibrationDetailedState.callback = None
comcam.rem.cccamera.evt_shutterDetailedState.callback = None
comcam.rem.cccamera.evt_endShutterOpen.callback = None
comcam.rem.cccamera.evt_startReadout.callback = None
comcam.rem.cccamera.evt_raftsDetailedState.callback = None

# Set Filter Tests

---
## 10: Set Filter
Send a CCCamera_command_setFilter command for positionFilter1.

Note: The filter being set must be one that was returned by the CCCamera_logevent_availableFilters event and cannot be the filter that is already set.
    
    Results:
    The CCCamera_logevent_startSetFilter event is published to the EFD with the name of the filter being set
    The CCCamera_logevent_startLoadFilter event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD showing UNLOADED
    The CCCamera_logevent_endLoadFilter event is published to the EFD
    The CCCamera_logevent_startRotateCarousel event is published to the EFD
    The CCCamera_logevent_endRotatorCarousel event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD and showing LOADING
    The CCCamera_logevent_startLoadFilter event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD showing LOADED
    The CCCamera_logevent_endSetFilter event is published to the EFD

In [35]:
await comcam.get_current_filter()

'r_03'

In [36]:
comcam.rem.cccamera.evt_startSetFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_startLoadFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_filterChangerDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_endLoadFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_startRotateCarousel.callback = evt_time_callback
comcam.rem.cccamera.evt_endRotateCarousel.callback = evt_time_callback
comcam.rem.cccamera.evt_endSetFilter.callback = evt_time_callback

In [37]:
await comcam.rem.cccamera.cmd_setFilter.set_start(name = 'i_06')


 startSetFilter logevent at 2022-08-10 17:09:32.985 is 
 	private_revCode: 031a81c3, private_sndStamp: 1660151335.9852293, private_rcvStamp: 1660151335.985533, private_seqNum: 1, private_identity: CCCamera, private_origin: 89351361, filterName: i_06, filterType: i

 filterChangerDetailedState logevent at 2022-08-10 17:09:32.988 is 
 	private_revCode: 00917c89, private_sndStamp: 1660151335.987648, private_rcvStamp: 1660151335.9879217, private_seqNum: 6, private_identity: CCCamera, private_origin: 89351361, substate: 1, timestampTransition: 0.0

 filterChangerDetailedState logevent at 2022-08-10 17:09:47.986 is 
 	private_revCode: 00917c89, private_sndStamp: 1660151350.9856045, private_rcvStamp: 1660151350.9858632, private_seqNum: 7, private_identity: CCCamera, private_origin: 89351361, substate: 4, timestampTransition: 0.0

 startRotateCarousel logevent at 2022-08-10 17:09:47.986 is 
 	private_revCode: dba0ed3b, private_sndStamp: 1660151350.9862163, private_rcvStamp: 1660151350.9864953

In [38]:
comcam.rem.cccamera.evt_startSetFilter.callback = None
comcam.rem.cccamera.evt_startLoadFilter.callback = None
comcam.rem.cccamera.evt_filterChangerDetailedState.callback = None
comcam.rem.cccamera.evt_endLoadFilter.callback = None
comcam.rem.cccamera.evt_startRotateCarousel.callback = None
comcam.rem.cccamera.evt_endRotateCarousel.callback = None
comcam.rem.cccamera.evt_endSetFilter.callback = None

In [39]:
await comcam.get_current_filter()

'i_06'

---
## 11: Set Filter to None 
Send a CCCamera_command_setFilter command with NONE.

Note: This indicates that the loaded filter should be unloaded. However, this will only work if one of the filter positions is EMPTY. If there are no filter positions that are empty, we expect this step to fail.

    Results:
    The CCCamera_logevent_startSetFilter event is published to the EFD with the name of the filter being set
    The CCCamera_logevent_startUnloadFilter event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD showing UNLOADING
    The CCCamera_logevent_endUnloadFilter event is published to the EFD
    The CCCamera_logevent_filterChangerDetailedState event is published to the EFD 
    The CCCamera_logevent_endSetFilter event is published to the EFD

In [40]:
comcam.rem.cccamera.evt_startSetFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_startUnloadFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_filterChangerDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_endUnloadFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_endSetFilter.callback = evt_time_callback

In [41]:
await comcam.rem.cccamera.cmd_setFilter.set_start(name = 'None')

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1520055160, ack=<SalRetCode.CMD_NOPERM: -300>, error=0, result='Ack : NO Command rejected: Invalid filter: None')

In [42]:
comcam.rem.cccamera.evt_startSetFilter.callback = None
comcam.rem.cccamera.evt_startUnloadFilter.callback = None
comcam.rem.cccamera.evt_filterChangerDetailedState.callback = None
comcam.rem.cccamera.evt_endUnloadFilter.callback = None
comcam.rem.cccamera.evt_endSetFilter.callback = None

# Take Image test

--- 
## 12: Init Image
While in the ENABLED state, send a CCCamera_command_initImage command of 15s.

Note: This deltaT value is an estimate of time before sending the CCCamera_command_takeImage command. If the command can be sent sooner/later, adjust the deltaT value.

    Results: 
    The command is accepted
    The CCCamera_logevent_ImageReadinesDetailedState event is published to the EFD showing the system transitions into the Getting_Ready state
    A CCCamera_logevent_prepareToTakeImage event is published to the EFD at the time of the transition
    A CCCamera_logevent_NotReadyToTakeImage event is published before reaching the Ready state.
    
    
**** Not available at EFD `comcam.rem.cccamera.evt_prepareToTakeImage` & `comcam.rem.cccamera.evt_notReadyToTakeImage`

In [43]:
comcam.rem.cccamera.evt_imageReadinessDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_prepareToTakeImage.callback = evt_time_callback
comcam.rem.cccamera.evt_notReadyToTakeImage.callback = evt_time_callback

In [44]:
await comcam.rem.cccamera.cmd_initImage.set_start(deltaT = 15)


 imageReadinessDetailedState logevent at 2022-08-10 17:10:32.943 is 
 	private_revCode: 90d66834, private_sndStamp: 1660151395.9426088, private_rcvStamp: 1660151395.9429963, private_seqNum: 0, private_identity: CCCamera, private_origin: 89351361, substate: 1, timestampTransition: 0.0

 imageReadinessDetailedState logevent at 2022-08-10 17:10:32.943 is 
 	private_revCode: 90d66834, private_sndStamp: 1660151395.9432936, private_rcvStamp: 1660151395.9436, private_seqNum: 1, private_identity: CCCamera, private_origin: 89351361, substate: 3, timestampTransition: 0.0


In [45]:
comcam.rem.cccamera.evt_imageReadinessDetailedState.callback = None
comcam.rem.cccamera.evt_prepareToTakeImage.callback = None
comcam.rem.cccamera.evt_notReadyToTakeImage.callback = None

---
##  13: Ready to take image.  
Verify the Camera is ready to take a picture.
    
    Results:
    The CCCamera_logevent_ImageReadinesDetailedState event is published to the EFD showing the system transitions into the Ready state
    A CCCamera_logevent_readyToTakeImage event is published to the EFD at the time of the transition
    A CCCamera_logevent_endInitializeImage event is published to the EFD at the time of the transition
    
**** Not available at EFD `comcam.rem.cccamera.evt_readyToTakeImage` & `comcam.rem.cccamera.evt_endInitializeImage`

In [46]:
readiness = comcam.rem.cccamera.evt_imageReadinessDetailedState.get()
time_readiness = Time(readiness.private_sndStamp, format="unix", scale="tai")
time_readiness.format = "iso"
logger.info(f'Image Readiness Detailed State at {time_readiness}' 
            f' is \n {readiness}')

In [47]:
ready_to_take_image = comcam.rem.cccamera.evt_readyToTakeImage.get()
time_ready_to_take_image = Time(ready_to_take_image.private_sndStamp, format="unix", scale="tai")
time_ready_to_take_image.format = "iso"
logger.info(f'Ready to take image at {time_ready_to_take_image}' 
            f' is \n {ready_to_take_image}')


AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

In [48]:
end_initialize_image = comcam.rem.cccamera.evt_endInitializeImage.get()
time_end_initialize_image = Time(end_initialize_image.private_sndStamp, format="unix", scale="tai")
time_end_initialize_image.format = "iso"
logger.info(f'End Initialize Image at {time_end_initialize_image}' 
            f' is \n {end_initialize_image}')

AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

---
## 14: Take Images
With the Camera ready to take a picture, send the CCCamera_command_takeImages command.

    Results:
    A CCCamera_logevent_startIntegration event is published to the EFD
    A CCCamera_logevent_startShutterOpen event is published to the EFD.
    A CCCamera_logevent_endShutterOpen event is published to the EFD.
    The Camera publishes a CCCamera_logevent_startShutterClose event as the camera shutter starts to shut
    The Camera publishes a CCCamera_logevent_endShutterClose event when the camera shutter is closed
    A CCCamera_logevent_startReadout event is published
    A CCCamera_logevent_endReadout event is published

In [49]:
comcam.rem.cccamera.evt_startIntegration.callback = evt_time_callback
comcam.rem.cccamera.evt_startShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_startShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_startReadout.callback = evt_time_callback
comcam.rem.cccamera.evt_endReadout.callback = evt_time_callback

In [50]:
keyValueMap = f'groupId:{Time.now().isot},imageType:ENGTEST'

In [51]:
# Set parameters
comcam.rem.cccamera.cmd_takeImages.set(shutter=True, keyValueMap=keyValueMap, numImages=1, expTime=20, obsNote = 'Test')

True

In [52]:
await comcam.rem.cccamera.cmd_takeImages.start()


 startIntegration logevent at 2022-08-10 17:11:13.278 is 
 	private_revCode: ac7ce4ef, private_sndStamp: 1660151436.278296, private_rcvStamp: 1660151436.2786934, private_seqNum: 3, private_identity: CCCamera, private_origin: 89351361, additionalKeys: groupId:imageType, additionalValues: 2022-08-10T17\:09\:55.034:ENGTEST, imagesInSequence: 1, imageName: CC_O_20220810_000002, imageIndex: 1, imageSource: CC, imageController: O, imageDate: 20220810, imageNumber: 2, timestampAcquisitionStart: 1660151304.3935397, exposureTime: 20.0, mode: normal, timeout: 20.0

 startShutterOpen logevent at 2022-08-10 17:11:13.279 is 
 	private_revCode: e26069d4, private_sndStamp: 1660151436.2793167, private_rcvStamp: 1660151436.2796803, private_seqNum: 1, private_identity: CCCamera, private_origin: 89351361, timestampTransition: 1660151436.2781088

 endShutterOpen logevent at 2022-08-10 17:11:14.260 is 
 	private_revCode: dc9d3865, private_sndStamp: 1660151437.2602115, private_rcvStamp: 1660151437.260584, 


 endReadout logevent at 2022-08-10 17:11:38.346 is 
 	private_revCode: 82211edd, private_sndStamp: 1660151461.345527, private_rcvStamp: 1660151461.3459034, private_seqNum: 3, private_identity: CCCamera, private_origin: 89351361, additionalKeys: groupId:imageType, additionalValues: 2022-08-10T17\:09\:55.034:ENGTEST, imagesInSequence: 1, imageName: CC_O_20220810_000002, imageIndex: 1, imageSource: CC, imageController: O, imageDate: 20220810, imageNumber: 2, timestampAcquisitionStart: 1660151304.3935397, requestedExposureTime: 20.0, timestampEndOfReadout: 1660151461.3436


In [53]:
comcam.rem.cccamera.evt_startIntegration.callback = None
comcam.rem.cccamera.evt_startShutterOpen.callback = None
comcam.rem.cccamera.evt_endShutterOpen.callback = None
comcam.rem.cccamera.evt_startShutterClose.callback = None
comcam.rem.cccamera.evt_endShutterClose.callback = None
comcam.rem.cccamera.evt_startReadout.callback = None
comcam.rem.cccamera.evt_endReadout.callback = None

---
## 15: Camera is done taking images 
Verify the Camera is done taking pictures.

    Results:
    A CCCamera_logevent_endTakeImage event is published to the EFD at the time the state transition occurs.
    A CCCamera_logevent_endOfImageTelemetry event is published
    
**** Not available at EFD `comcam.rem.cccamera.evt_endTakeImage` 

In [54]:
end_take_image = comcam.rem.cccamera.evt_endTakeImage.get()
time_end_take_image = Time(end_take_image.private_sndStamp, format="unix", scale="tai")
time_end_take_image.format = "iso"
logger.info(f'End Take Image at {time_end_take_image}' 
            f' is \n {end_take_image}')

AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

In [55]:
end_of_image_telemetry = comcam.rem.cccamera.evt_endOfImageTelemetry.get()
time_end_of_image_telemetry = Time(end_of_image_telemetry.private_sndStamp, format="unix", scale="tai")
time_end_of_image_telemetry.format = "iso"
logger.info(f'End of Image Telemetry at {time_end_of_image_telemetry}' 
            f' is \n {end_of_image_telemetry}')

# Guiders Image Test

---
## 16: Init Image

While in the ENABLED state, send a CCCamera_command_initImage command of 15s.

    Results:
    The command is accepted
    The CCCamera_logevent_ImageReadinesDetailedState event is published to the EFD showing the system transitions into the Getting_Ready state
    A CCCamera_logevent_prepareToTakeImage event is published to the EFD at the time of the transition
    A CCCamera_logevent_NotReadyToTakeImage event is published before reaching the Ready state.

In [56]:
comcam.rem.cccamera.evt_imageReadinessDetailedState.callback = evt_time_callback
comcam.rem.cccamera.evt_prepareToTakeImage.callback = evt_time_callback
comcam.rem.cccamera.evt_notReadyToTakeImage.callback = evt_time_callback

In [57]:
await comcam.rem.cccamera.cmd_initImage.set_start(deltaT = 15)


 imageReadinessDetailedState logevent at 2022-08-10 17:12:28.409 is 
 	private_revCode: 90d66834, private_sndStamp: 1660151511.4090407, private_rcvStamp: 1660151511.4093182, private_seqNum: 2, private_identity: CCCamera, private_origin: 89351361, substate: 1, timestampTransition: 0.0

 imageReadinessDetailedState logevent at 2022-08-10 17:12:28.410 is 
 	private_revCode: 90d66834, private_sndStamp: 1660151511.4099727, private_rcvStamp: 1660151511.410178, private_seqNum: 3, private_identity: CCCamera, private_origin: 89351361, substate: 3, timestampTransition: 0.0


In [58]:
comcam.rem.cccamera.evt_imageReadinessDetailedState.callback = None
comcam.rem.cccamera.evt_prepareToTakeImage.callback = None
comcam.rem.cccamera.evt_notReadyToTakeImage.callback = None

---
## 17: Ready to take image 
Verify the Camera is ready to take a picture.

    Results:
    The CCCamera_logevent_ImageReadinesDetailedState event is published to the EFD showing the system transitions into the Ready state
    A CCCamera_logevent_readyToTakeImage event is published to the EFD at the time of the transition
    A CCCamera_logevent_endInitializeImage event is published to the EFD at the time of the transition

In [59]:
readiness = comcam.rem.cccamera.evt_imageReadinessDetailedState.get()
time_readiness = Time(readiness.private_sndStamp, format="unix", scale="tai")
time_readiness.format = "iso"
logger.info(f'Image Readiness Detailed State at {time_readiness}' 
            f' is \n {readiness}')

In [60]:
ready_to_take_image = comcam.rem.cccamera.evt_readyToTakeImage.get()
time_ready_to_take_image = Time(ready_to_take_image.private_sndStamp, format="unix", scale="tai")
time_ready_to_take_image.format = "iso"
logger.info(f'Ready to take image at {time_ready_to_take_image}' 
            f' is \n {ready_to_take_image}')


AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

In [61]:
end_initialize_image = comcam.rem.cccamera.evt_endInitializeImage.get()
time_end_initialize_image = Time(end_initialize_image.private_sndStamp, format="unix", scale="tai")
time_end_initialize_image.format = "iso"
logger.info(f'End Initialize Image at {time_end_initialize_image}' 
            f' is \n {end_initialize_image}')

AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

---
## 18: Init Guider 
Send a CCCamera_command_initGuider command.

Note: Guider Implementation is not complete and so the use of this command at the moment will result in a fail.

    Results:
    An CCCamera_logevent_endInitializeGuider event is published to the EFD.

**** Missing at EFD `comcam.rem.cccamera.evt_endInitializeGuider`

In [62]:
comcam.rem.cccamera.evt_endInitializeGuider.callback = evt_time_callback

In [65]:
await comcam.rem.cccamera.cmd_initGuiders.set_start()

In [64]:
comcam.rem.cccamera.evt_endInitializeGuider.callback = None

---
## 19: Take Images
With the Camera ready to take a picture, send the CCCamera_command_takeImages command.

    Results:
    A CCCamera_logevent_startIntegration event is published to the EFD
    A CCCamera_logevent_startShutterOpen event is published to the EFD.
    A CCCamera_logevent_endShutterOpen event is published to the EFD.
    The Camera publishes a CCCamera_logevent_startShutterClose event as the camera shutter starts to shut
    The Camera publishes a CCCamera_logevent_endShutterClose event when the camera shutter is closed
    A CCCamera_logevent_startReadout event is published
    A CCCamera_logevent_endReadout event is published

In [66]:
comcam.rem.cccamera.evt_startIntegration.callback = evt_time_callback
comcam.rem.cccamera.evt_startShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterOpen.callback = evt_time_callback
comcam.rem.cccamera.evt_startShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_endShutterClose.callback = evt_time_callback
comcam.rem.cccamera.evt_startReadout.callback = evt_time_callback
comcam.rem.cccamera.evt_endReadout.callback = evt_time_callback

In [67]:
keyValueMap = f'groupId:{Time.now().isot},imageType:ENGTEST'

In [68]:
# Set parameters
comcam.rem.cccamera.cmd_takeImages.set(shutter=True, keyValueMap=keyValueMap, numImages=1, expTime=20, obsNote= 'test')

True

In [69]:
await comcam.rem.cccamera.cmd_takeImages.start()


 startIntegration logevent at 2022-08-10 17:13:40.565 is 
 	private_revCode: ac7ce4ef, private_sndStamp: 1660151583.5650425, private_rcvStamp: 1660151583.5652077, private_seqNum: 4, private_identity: CCCamera, private_origin: 89351361, additionalKeys: groupId:imageType, additionalValues: 2022-08-10T17\:12\:23.913:ENGTEST, imagesInSequence: 1, imageName: CC_O_20220810_000003, imageIndex: 1, imageSource: CC, imageController: O, imageDate: 20220810, imageNumber: 3, timestampAcquisitionStart: 1660151461.3417122, exposureTime: 20.0, mode: normal, timeout: 20.0

 startShutterOpen logevent at 2022-08-10 17:13:40.566 is 
 	private_revCode: e26069d4, private_sndStamp: 1660151583.5663595, private_rcvStamp: 1660151583.5665183, private_seqNum: 2, private_identity: CCCamera, private_origin: 89351361, timestampTransition: 1660151583.5650525

 endShutterOpen logevent at 2022-08-10 17:13:41.547 is 
 	private_revCode: dc9d3865, private_sndStamp: 1660151584.5472245, private_rcvStamp: 1660151584.5475218

In [70]:
comcam.rem.cccamera.evt_startIntegration.callback = None
comcam.rem.cccamera.evt_startShutterOpen.callback = None
comcam.rem.cccamera.evt_endShutterOpen.callback = None
comcam.rem.cccamera.evt_startShutterClose.callback = None
comcam.rem.cccamera.evt_endShutterClose.callback = None
comcam.rem.cccamera.evt_startReadout.callback = None
comcam.rem.cccamera.evt_endReadout.callback = None

---
## 20: Camera is done taking images 
Verify the Camera is done taking pictures.

    Results:
    A CCCamera_logevent_endTakeImage event is published to the EFD at the time the state transition occurs.
    A CCCamera_logevent_endOfImageTelemetry event is published

In [71]:
end_take_image = comcam.rem.cccamera.evt_endTakeImage.get()
time_end_take_image = Time(end_take_image.private_sndStamp, format="unix", scale="tai")
time_end_take_image.format = "iso"
logger.info(f'End Take Image at {time_end_take_image}' 
            f' is \n {end_take_image}')

AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

In [72]:
end_of_image_telemetry = comcam.rem.cccamera.evt_endOfImageTelemetry.get()
time_end_of_image_telemetry = Time(end_of_image_telemetry.private_sndStamp, format="unix", scale="tai")
time_end_of_image_telemetry.format = "iso"
logger.info(f'End of Image Telemetry at {time_end_of_image_telemetry}' 
            f' is \n {end_of_image_telemetry}')

# Check Events in EFD

---
## 21: Image Visualization
Verify the CCCamera_logevent_imageVisualization event is not published into the EFD.    
Note: Although this is defined in the latest XML release, this event is not currently implemented and is not expected to be generated to the EFD.

In [73]:
image_visualization = comcam.rem.cccamera.evt_imageVisualization.get()
time_image_visualization = Time(image_visualization.private_sndStamp, format="unix", scale="tai")
time_image_visualization.format = "iso"
logger.info(f'Image Visualization at {time_image_visualization}' 
            f' is \n {image_visualization}')

AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

---
## 22: Image Stored 
Verify the CCCamera_logevent_imageStored event is not published into the EFD.
Note: Although this is defined in the latest XML release, this event is not currently implemented and is not expected to be generated to the EFD.

In [74]:
image_stored = comcam.rem.cccamera.evt_imageStored.get()
time_image_stored = Time(image_stored.private_sndStamp, format="unix", scale="tai")
time_image_stored.format = "iso"
logger.info(f'Image Stored at {time_image_stored}' 
            f' is \n {image_stored}')

AttributeError: 'NoneType' object has no attribute 'private_sndStamp'

# Stop Command

---
## 23: Set Filter


While in the ENABLED state, send CCCamera_command_setFilter command for positionFilter1.
    
    Results:
    The CCCamera_logevent_startSetFilter is published to the EFD with the name of the filter being set
    
    
The next step `24: Stop` should be run right after this one, so the first `setFilter` command can be stopped as it'll be run in the BG. 

In [85]:
await comcam.get_current_filter()

'i_06'

In [86]:
comcam.rem.cccamera.evt_startSetFilter.callback = evt_time_callback
comcam.rem.cccamera.evt_endSetFilter.callback = evt_time_callback

In [91]:
comcam.rem.cccamera.cmd_setFilter.set_start(name = 'r_03')

CancelledError: 

---
## 24: Stop 
Before an CCCamera_logevent_endSetFilter event is published, send a CCCamera_command_stop command.

Note: The CCCamera_command_setFilter command will stop once it is safe to do so. The CCCamera_command_stop command may not stop the filter from being set if the command was given while in an unsafe position.

    Results:
    The command is accepted
    The CCCamera_logevent_endSetFilter event shows that no filter has been set

In [92]:
comcam.rem.cccamera.cmd_stop

RemoteCommand(CCCamera, 0, stop)

In [89]:
comcam.rem.cccamera.evt_startSetFilter.callback = None
comcam.rem.cccamera.evt_endSetFilter.callback = None

In [94]:
await comcam.get_current_filter()

'i_06'

---
## 25: Start Image
While the CalibrationDetailedState is in ENABLED, send a CCCamera_command_startImage command with the following parameters:

                        shutter: True
                        sensors
                        keyValueMap
                        obsNote: Any arbitrary string
                        timeout: 30s

The next step `26: Stop` should be run right after this one, so the first `setFilter` command can be stopped as it'll be run in the BG. 

In [96]:
# Check CalibrationDetailedState is ENABLED
calibration_detailed_state= comcam.rem.cccamera.evt_calibrationDetailedState.get()
time_calibration_detailed_state=  Time(calibration_detailed_state.private_sndStamp, format="unix", scale="tai")
time_calibration_detailed_state.format = "iso"
logger.info(f'Calibration Detailed State at {time_calibration_detailed_state} are {calibration_detailed_state}')

In [97]:
keyValueMap = f'groupId:{Time.now().isot},imageType:ENGTEST'

In [98]:
# Set parameters
comcam.rem.cccamera.cmd_startImage.set(shutter=True, keyValueMap=keyValueMap, timeout=30, obsNote='Test')

True

In [99]:
# Start image
await comcam.rem.cccamera.cmd_startImage.start()

---
## 26: Stop 
Before the specified timeout, send a CCCamera_command_stop command.

    Results:
    The previous CCCamera_command_startImage command is abandoned.

In [100]:
# Check CalibrationDetailedState is ENABLED
start_integration= comcam.rem.cccamera.evt_startIntegration.get()
time_start_integration=  Time(start_integration.private_sndStamp, format="unix", scale="tai")
time_start_integration.format = "iso"
logger.info(f'Start Integration at {time_start_integration} are {start_integration}')

In [101]:
stop = comcam.rem.cccamera.cmd_stop
logger.info(f'{stop} at {Time.now()}')

In [103]:
end_of_image_telemetry = comcam.rem.cccamera.evt_endOfImageTelemetry.get()
time_end_of_image_telemetry = datetime.utcfromtimestamp(end_of_image_telemetry.private_sndStamp).strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
logger.info(f'End of Image Telemetry at {time_end_of_image_telemetry}' 
            f' is \n {end_of_image_telemetry}')

# Playlist Test - only valid when using emulated DAQ

---
## 27: Create playlist
Send a CCCamera_command_playList command of the playlist to be defined and the current folder of where the images exist.

    Results:
    The command is accepted and a playlist is created.

In [ ]:
await comcam.rem.cccamera.cmd_playlist.set_start(playlist='test.playlist', folder =  , images = :)

---
## 28: Play playlist in repeat 
Send a CCCamera_command_play coommand of the playlist and an input of true for the repeat parameter.
    
    Results:
    The command is accepted
    The playlist is played and repeats

In [ ]:
await comcam.rem.cccamera.cmd_play.set_start(playlist='tiago-1.playlist', repeat =True)

# Disable Calibration 

---
##  29: Disable Calibraiton
With the Summary State of the Camera in ENABLED and the CalibrationDetailedState in ENABLED, send a CCCamera_command_disableCalibration command.

    Results:
    The  CCCamera_logevent_CalibrationDetailedState event publishes DISABLED.

In [104]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = evt_time_callback

In [105]:
await comcam.rem.cccamera.cmd_disableCalibration.set_start()


 calibrationDetailedState logevent at 2022-08-10 17:22:18.979 is 
 	private_revCode: a610ea82, private_sndStamp: 1660152101.9792845, private_rcvStamp: 1660152101.9795775, private_seqNum: 8, private_identity: CCCamera, private_origin: 89351361, substate: 1, timestampTransition: 0.0


In [106]:
comcam.rem.cccamera.evt_calibrationDetailedState.callback = None

--- 
# EFD Events

---
## 31: Shutter Blade Motion Profile Event
Verify the CCCamera_logevent_ShutterBladeMotionProfile event is not published to the EFD.

Note: Although this is defined in the latest XML release, this event is not currently implemented and is not expected to be generated to the EFD.

In [107]:
await check_telemetry('lsst.sal.CCCamera.evt_shutterBladeMotionProfile')

AttributeError: 'DataFrame' object has no attribute 'private_efdStamp'

---
## 32: Fits Files Written Event
Verify the CCCamera_logevent_fitsFilesWritten event is not published to the EFD.

Note: Although this is defined in the latest XML release, this event is not currently implemented and is not expected to be generated to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.logevent_fitsFilesWritten')

---
## 32: File Command Execution Event
Verify the CCCamera_logevent_FileCommandExecution event is not published to the EFD.

Note: Although this is defined in the latest XML release, this event is not currently implemented and is not expected to be generated to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.logevent_FileCommandExecution')

# EFD Telemetry

---
## 33: FCS
Verify the CCCamera_fcs telemetry data is being published to the EFD.

In [108]:
await check_telemetry('lsst.sal.CCCamera.fcs')

AttributeError: 'DataFrame' object has no attribute 'private_efdStamp'

---
## 34: Bonn Shutter Device
Verify the CCCamera_bonn_shutter_Device telemetry data is being published to the EFD.

In [110]:
await check_telemetry('lsst.sal.CCCamera.bonn_shutter_Device')

AttributeError: 'DataFrame' object has no attribute 'private_efdStamp'

---
## 35: DAQ Monitor Store
Verify the CCCamera_daq_monitor_Store telemetry data is being published to the EFD.

In [111]:
await check_telemetry('lsst.sal.CCCamera.daq_monitor_Store')

Last lsst.sal.CCCamera.daq_monitor_Store telemetry at 2022-08-10 17:23:48.271 UTC (time now 2022-08-10 17:23:58.087605) was 

                                       capacity  freeFraction     freeSpace  \
2022-08-10 17:23:48.270542+00:00  12000.405273      0.959062  11509.126953   

                                  private_efdStamp private_identity  \
2022-08-10 17:23:48.270542+00:00      1.660152e+09         CCCamera   

                                  private_kafkaStamp  private_origin  \
2022-08-10 17:23:48.270542+00:00        1.660152e+09        89351361   

                                  private_rcvStamp private_revCode  \
2022-08-10 17:23:48.270542+00:00      1.660152e+09        41517a25   

                                  private_seqNum  private_sndStamp  
2022-08-10 17:23:48.270542+00:00            8706      1.660152e+09  


---
## 36: Rebpower REB
Verify the CCCamera_rebpower_Reb telemetry data is being published to the EFD.

In [112]:
await check_telemetry('lsst.sal.CCCamera.rebpower_Reb')

AttributeError: 'DataFrame' object has no attribute 'private_efdStamp'

---
## 37: Rebpower REBPS 
Verify the CCCamera_rebpower_Rebps telemetry data is being published to the EFD.

In [113]:
await check_telemetry('lsst.sal.CCCamera.rebpower_Rebps')

AttributeError: 'DataFrame' object has no attribute 'private_efdStamp'

---
## 38: Vacumm Cold1
Verify the CCCamera_vacuum_Cold1 telemetry data is being published to the EFD.

In [114]:
await check_telemetry('lsst.sal.CCCamera.vacuum_Cold1')

AttributeError: 'DataFrame' object has no attribute 'private_efdStamp'

---
## 39: Vacuum Cold2 
Verify the CCCamera_vacuum_Cold2 telemetry data is being published to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.vacuum_Cold2')

---
## 40: Vacuum Cryo 
Verify the CCCamera_vacuum_Cryo telemetry data is being published to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.vacuum_Cryo')

---
## 41: Vacuum IonPumps
Verify the CCCamera_vacuum_IonPumps telemetry data is being published to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.vacuum_IonPumps')

---
## 42: Vacuum RTDS
Verify the CCCamera_vacuum_Rtds telemetry data is being published to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.vacuum_Rtds')

---
## 43: Vacuum Turbo
Verify the CCCamera_vacuum_Turbo telemetry data is being published to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.vacuum_Turbo')

---
## 44: Vacuum VQMonitor
Verify the CCCamera_vacuum_VQMonitor telemetry data is being published to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.vacuum_VQMonitor')

---
## 45: Quadbox BFR
Verify the CCCamera_quadbox_BFR telemetry data is being published to the EFD.

In [115]:
await check_telemetry('lsst.sal.CCCamera.quadbox_BFR')

AttributeError: 'DataFrame' object has no attribute 'private_efdStamp'

---
## 46: Quadbox PDU 24VD 
Verify the CCCamera_quadbox_PDU_24VD telemetry data is being published to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.quadbox_PDU_24VD')

---
## 47: Quadbox PDF 24VDC
Verify the CCCamera_quadbox_PDU_24VDC telemetry data is being published to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.quadbox_PDU_24VC')

---
## 48: Quadbox PDU 48V 
Verify the CCCamera_quadbox_PDU_48V telemetry data is being published to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.quadbox_PDU_48V')

---
## 49: Quadbox PDU 5V 
Verify the CCCamera_quadbox_PDU_5V telemetry data is being published to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.quadbox_PDU_5V')

---
## 50: Focal Place CCD 
Verify the CCCamera_focal_plane_Ccd telemetry data is being published to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.focal_plane_Ccd')

---
## 51: Focal Plane Reb
Verify the CCCamera_focal_plane_Reb telemetry data is being published to the EFD.

In [116]:
await check_telemetry('lsst.sal.CCCamera.focal_plane_Reb')

Last lsst.sal.CCCamera.focal_plane_Reb telemetry at 2022-08-10 17:24:26.443 UTC (time now 2022-08-10 17:24:28.003491) was 

                                  anaI0 anaI1 anaI2 anaV0 anaV1 anaV2  \
2022-08-10 17:24:26.442640+00:00  None  None  None  None  None  None   

                                 aspicl_Temp00  aspicl_Temp01 aspicl_Temp02  \
2022-08-10 17:24:26.442640+00:00          None     442.321442          None   

                                 aspicl_Temp10  ...  temp62 temp70 temp71  \
2022-08-10 17:24:26.442640+00:00          None  ...    None   None   None   

                                  temp72 temp80 temp81  temp82 temp90 temp91  \
2022-08-10 17:24:26.442640+00:00    None   None   None    None   None   None   

                                  temp92  
2022-08-10 17:24:26.442640+00:00    None  

[1 rows x 138 columns]


---
## 52: Focal Plane Reb Total Power
Verify the CCCamera_focal_plane_RebTotalPower telemetry data is being published to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.focal_plane_RebTotalPower')

---
## 53: Focal Plane Segment
Verify the CCCamera_focal_plane_Segment telemetry data is being published to the EFD.

In [ ]:
await check_telemetry('lsst.sal.CCCamera.focal_plane_Segment')

In [117]:
script.log.info(f"STOP- {test_message}")